In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

train = pd.read_csv('/kaggle/input/hackethon-summer/Train_Data (1).csv') 
test = pd.read_csv('/kaggle/input/hackethon-summer/Test_Data (1).csv') 

numerical_cols = ['BMXBMI', 'LBXGLU', 'LBXGLT', 'LBXIN']
cat_cols = ['RIAGENDR', 'PAQ605', 'DIQ010']
target = 'age_group'

train = train.dropna(subset=[target])
train.drop(columns = 'SEQN', inplace = True)
test.drop(columns = 'SEQN', inplace = True) 


In [2]:
test

,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,1.0,1.0,32.2,96.0,2.0,135.0,15.11
1,2.0,2.0,26.3,100.0,2.0,141.0,15.26
2,1.0,2.0,28.6,107.0,2.0,136.0,8.82
3,2.0,1.0,22.1,93.0,2.0,111.0,12.13
4,1.0,1.0,24.7,91.0,2.0,105.0,3.12
...,...,...,...,...,...,...,...
307,2.0,2.0,21.9,82.0,2.0,82.0,2.54
308,2.0,1.0,33.3,95.0,2.0,77.0,6.36
309,2.0,2.0,41.5,91.0,2.0,149.0,15.52
310,2.0,2.0,22.5,82.0,2.0,93.0,1.39


In [3]:
le = LabelEncoder()
train[target] = le.fit_transform(train[target])

for col in cat_cols:
    train[col].fillna(train[col].mode()[0], inplace = True)
    test[col].fillna(test[col].mode()[0], inplace = True)

for col in numerical_cols:
    train[col].fillna(train[col].median(), inplace = True)
    test[col].fillna(test[col].median(), inplace = True)

/tmp/ipykernel_13/931257579.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace = True)
/tmp/ipykernel_13/931257579.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [4]:
train

,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,0
1,2.0,2.0,20.3,89.0,2.0,80.0,3.85,0
2,1.0,2.0,23.2,89.0,2.0,68.0,6.14,0
3,1.0,2.0,28.9,104.0,2.0,84.0,16.15,0
4,2.0,1.0,35.9,103.0,2.0,81.0,10.92,0
...,...,...,...,...,...,...,...,...
1961,2.0,2.0,33.5,100.0,2.0,73.0,6.53,0
1962,1.0,2.0,30.0,93.0,2.0,208.0,13.02,0
1963,1.0,2.0,23.7,103.0,2.0,124.0,21.41,0
1964,2.0,2.0,27.4,90.0,2.0,108.0,4.99,0


In [5]:

# Prepare data for mutual information
X = train.drop(columns= target)
y = train[target]

#Compute Mutual Information scores
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Print ranked feature importance
print("Feature Importance (Mutual Information):")
print(mi_series)

# Plot top 5 features against target
top_5_features = mi_series.head(5).index.tolist()


Feature Importance (Mutual Information):
LBXGLT      0.044295
PAQ605      0.011489
LBXIN       0.010960
LBXGLU      0.004083
RIAGENDR    0.001979
BMXBMI      0.001530
DIQ010      0.000000
dtype: float64


In [6]:
# for feature in top_5_features:
#     plt.figure(figsize=(6, 4))
#     if train[feature].nunique() <= 10:
#         sns.countplot(x=feature, hue=target, data=train)
#     else:
#         sns.boxplot(x=target, y=feature, data=train)
#     plt.title(f'{feature} vs Age Group')
#     plt.xlabel(feature)
#     plt.ylabel('Value')
#     plt.legend(title='Age Group')
#     plt.tight_layout()
#     plt.show()

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [8]:
# Train Random Forest using top 5 features
X_model = train[top_5_features]
y_model = train[target]

X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.3, random_state = 40, stratify=y)



In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 2000),
    'max_depth': [None] + list(range(5, 50, 5)),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'class_weight': ['balanced', None]
}

clf = RandomForestClassifier(random_state=42)

rand_search = RandomizedSearchCV(
    clf, param_distributions=param_dist,
    n_iter=50, scoring='f1_macro', cv=5,
    verbose=2, random_state=42, n_jobs=-1
)

rand_search.fit(X_train, y_train)
print("Best Parameters:", rand_search.best_params_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END class_weight=balanced, max_depth=15, max_features=sqrt, min_samples_leaf=8, min_samples_split=6, n_estimators=1144; total time=   3.9s
[CV] END class_weight=balanced, max_depth=45, max_features=None, min_samples_leaf=7, min_samples_split=4, n_estimators=1582; total time=   7.2s
[CV] END class_weight=None, max_depth=20, max_features=None, min_samples_leaf=6, min_samples_split=6, n_estimators=869; total time=   4.0s
[CV] END class_weight=None, max_depth=20, max_features=None, min_samples_leaf=6, min_samples_split=6, n_estimators=869; total time=   3.9s
[CV] END class_weight=None, max_depth=25, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=559; total time=   2.0s
[CV] END class_weight=None, max_depth=25, max_features=sqrt, min_samples_leaf=9, min_samples_split=2, n_estimators=574; total time=   2.0s
[CV] END class_weight=balanced, max_depth=45, max_features=None, min_samples_leaf=3, min_sampl

In [10]:
clf = RandomForestClassifier(
    class_weight = 'balanced', 
    max_depth = 40, 
    max_features = 'log2', 
    min_samples_leaf = 5, 
    min_samples_split = 2, 
    n_estimators = 1942,
)
clf.fit(X_train, y_train)



print_score(clf, X_train, y_train, X_test, y_test, train=False)

Test Result:
Accuracy Score: 77.13%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.884120   0.333333  0.771331    0.608727      0.795769
recall       0.837398   0.425532  0.771331    0.631465      0.771331
f1-score     0.860125   0.373832  0.771331    0.616979      0.782119
support    492.000000  94.000000  0.771331  586.000000    586.000000
_______________________________________________
Confusion Matrix: 
 [[412  80]
 [ 54  40]]



In [11]:
from sklearn.svm import SVC
svc_clf = SVC(C=1000)
svc_clf.fit(X_train,y_train)

print_score(svc_clf, X_train, y_train, X_test, y_test, train=False)

Test Result:
Accuracy Score: 83.79%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.851327   0.476190  0.837884    0.663759      0.791152
recall       0.977642   0.106383  0.837884    0.542013      0.837884
f1-score     0.910123   0.173913  0.837884    0.542018      0.792028
support    492.000000  94.000000  0.837884  586.000000    586.000000
_______________________________________________
Confusion Matrix: 
 [[481  11]
 [ 84  10]]



In [12]:
from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression(max_iter=1000)
lg_clf.fit(X_train, y_train)

print_score(lg_clf, X_train, y_train, X_test, y_test, train=False)

Test Result:
Accuracy Score: 83.28%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.845614   0.375000  0.832765    0.610307      0.770123
recall       0.979675   0.063830  0.832765    0.521752      0.832765
f1-score     0.907721   0.109091  0.832765    0.508406      0.779613
support    492.000000  94.000000  0.832765  586.000000    586.000000
_______________________________________________
Confusion Matrix: 
 [[482  10]
 [ 88   6]]



In [13]:

X_test_data = test[top_5_features]
test_predictions = clf.predict(X_test_data)

# Submission
submission = pd.DataFrame({'age_group': test_predictions})
submission.to_csv("submission.csv", index=False)

submission.value_counts()

age_group
0            261
1             51
Name: count, dtype: int64